In [14]:
import os
import inspect
import seaborn
import matplotlib
import matplotlib.pyplot as plt
import torch
import scanpy as sc
import tqdm
import sys
import pickle
import PyComplexHeatmap as pch
import scvi
import IPython
import pandas as pd
import scipy
from scipy.stats import mannwhitneyu

sc.settings.figdir=os.path.expanduser('~/WbFigures/DevDE')
sc._settings.settings._vector_friendly=True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print("GPU is available")
    print("Number of GPUs:", torch.cuda.device_count())
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available")

sys.path.append('/home/matthew.schmitz/utils/mts-utils/')
from genomics import sc_analysis


import antipode
from antipode.antipode_model import *
import antipode.model_functions
from antipode.model_functions import *
import antipode.model_distributions
from antipode.model_distributions import *
import antipode.model_modules
from antipode.model_modules import *
import antipode.train_utils
from antipode.train_utils import *
import antipode.plotting
from antipode.plotting import *

gs1=matplotlib.colors.ListedColormap(sc.pl.palettes.godsnot_102,name='godsnot_102')
try:
    matplotlib.colormaps.register(name='godsnot_102',cmap=gs1)
except:
    pass


plt.rc("axes.spines", top=False, right=False)


/home/matthew.schmitz/Matthew/utils/miniforge3/envs/rapids-24.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU is not available


In [2]:
batch_key='batch_name'
discov_key='species'
layer_key='spliced'
leaf_key='Initial_Class_markers_level_2'
MDE_KEY = "X_antipode_UMAP"
anndata.settings.allow_write_nullable_strings = True

model_tag   = '1.9.1.8.5_Dev_final_600clusters'
model_path='/home/matthew.schmitz/Matthew/models/'+model_tag
adata = sc.read_h5ad(os.path.join(model_path,'analysis_inprogress.h5ad'))
antipode_model=antipode.antipode_model.ANTIPODE.load(model_path,adata=adata,prefix='p3_',device=device)
antipode_model.save_params_to_uns()
adata=antipode_model.adata_manager.adata
pstore = adata.uns['param_store']
adata.var = pd.read_csv('/home/matthew.schmitz/Matthew/data/taxtest/gene_lists/var.csv',index_col=0)

INFO     Generating sequential column names                                                                        


/home/matthew.schmitz/Matthew/utils/miniforge3/envs/rapids-24.12/lib/python3.12/site-packages/pyro/params/param_store.py:334: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  s

In [4]:
leaf_l2_dict = dict(zip(adata.obs[leaf_key].cat.categories,adata.obs['level_2'].cat.categories))
l2_leaf_dict = dict(zip(adata.obs['level_2'].cat.categories,adata.obs[leaf_key].cat.categories))

pstore = adata.uns['param_store']
discov_cluster_params,cluster_params, cluster_labels,var_labels, zero_mask,(prop_taxon, prop_locs,prop_discov_di,prop_discov_dm)=antipode_model.calculate_cluster_params(flavor='numpy')
keep_clusters = np.array([x in adata.obs['level_2'].unique() for x in cluster_labels ])
cluster_labels = np.array(cluster_labels)[keep_clusters]
zero_mask = zero_mask[:,keep_clusters,:]
cluster_index = np.array([int(x) for x in cluster_labels])
zero_mask = np.nan_to_num(zero_mask,posinf=np.nan)
discov_cluster_params = discov_cluster_params[:,cluster_index,:] * zero_mask
prop_locs = prop_locs[cluster_index,:]
prop_discov_di = prop_discov_di[:,cluster_index,:] * zero_mask
prop_discov_dm = prop_discov_dm[:,cluster_index,:] * zero_mask
discov_labels=adata.obs[antipode_model.discov_key].cat.categories
latent_labels=[str(x) for x in range(pstore['discov_dc'].shape[1])]
n_clusters=antipode_model.level_sizes[-1]
leaf_labels = [l2_leaf_dict[x] for x in cluster_labels]

/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/code/scANTIPODE/antipode/antipode_mixins.py:450: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  zero_mask = (adata.obs.groupby(self.discov_key)[leaf_level].value_counts().unstack().loc[:,cluster_labels]>=cluster_count_threshold).to_numpy()
/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/code/scANTIPODE/antipode/antipode_mixins.py:451: RuntimeWarning: divide by zero encountered in divide
  return discov_cluster_params,cluster_params, cluster_labels,var_labels, 1/zero_mask[...,np.newaxis],(prop_taxon, prop_locs,prop_discov_di,prop_discov_dm)


In [6]:
list(adata.obs.columns)

['Class',
 'G2M_score',
 'S_score',
 'Subclass',
 'batch_name',
 'female',
 'general_region',
 'log10_n_counts',
 'new_ccl',
 'new_neighborhood_extended',
 'phase',
 'region',
 'species',
 'timepoint',
 'general_region_species',
 '_scvi_discov_ind',
 '_scvi_batch_ind',
 'psi_0',
 'psi_1',
 'psi_2',
 'q_score',
 'level_0',
 'level_1',
 'level_2',
 'antipode_cluster',
 'kmeans',
 'clean_cellname',
 'dataset_name',
 'doublet',
 'full_cellname',
 'individual',
 'msregion',
 'n_counts',
 'n_genes',
 'percent_ribo',
 'singlet',
 'species_dataset_name',
 'anno_level_2',
 'initial_class',
 'ic_extended',
 'abc_subclass_markercor',
 'juv_subclass_markercor',
 'linnarson_extended',
 'lamanno_class',
 'lamanno_subclass',
 'lamanno_cluster',
 'neighborhood',
 'curated_class_label',
 'new_neighborhood',
 'ccl_extended',
 'Neighborhood',
 'Division',
 'Initial_Class',
 'Initial_Class_markers',
 'Initial_Class_markers_level_2',
 'clipped_psi_2',
 'log_abs_sum_dynam',
 'log10_n_genes']

In [107]:
keep_columns = [
 'G2M_score',
 'S_score',
 'batch_name',
 'general_region',
 'log10_n_counts',
 'log10_n_genes',
 'phase',
 'region',
 'species',
 'timepoint',
 'psi_1',
 'psi_2',
 'level_0',
 'level_1',
 'level_2',
 'antipode_cluster',
 'kmeans',
 'clean_cellname',
 'doublet',
 'full_cellname',
 'individual',
 'msregion',
 'percent_ribo',
 'species_dataset_name',
 'abc_subclass_markercor',
 'Neighborhood',
 'Division',
 'Initial_Class',
 'Initial_Class_markers',
 'Initial_Class_markers_level_2',
]

In [77]:
meta_columns = [
 'batch_name',
 'general_region',
 'region',
 'species',
 'timepoint',
 'individual',
 'species_dataset_name'
]


In [38]:
adata.obs['individual'] = adata.obs['individual'].astype(str)
adata.obs.loc[adata.obs['individual']=='nan','individual'] = adata.obs.loc[adata.obs['individual']=='nan','batch_name']

In [46]:
adata.obs.loc[adata.obs['species_dataset_name']=='mouse:PRJNA641653','general_region'] = 'ctx'

In [42]:
ribo_genes=[name for name in adata.var_names if name.startswith('RPS') or name.startswith('RPL') ]
adata.obs['percent_ribo'] = np.sum(
adata[:, ribo_genes].layers['spliced'], axis=1) / np.sum(adata.X, axis=1)
mito_genes = [name for name in adata.var_names if name in ['ND1','ND2','ND4L','ND4','ND5','ND6','ATP6','ATP8','CYTB','COX1','COX2','COX3'] or name.startswith('CHRM-') or name.startswith('MT-')]
adata.obs['percent_mito'] = np.sum(
adata[:, mito_genes].layers['spliced'], axis=1) / np.sum(adata.X, axis=1)


In [35]:
# import anndata
# import h5py
# with h5py.File(os.path.join(model_path,'analysis_inprogress.h5ad')) as f:
#     obs = anndata.io.read_elem(f['obs'])
# adata.obs = obs

In [62]:
adata.obs[keep_columns]

,G2M_score,S_score,batch_name,general_region,log10_n_counts,log10_n_genes,phase,region,species,timepoint,...,individual,msregion,percent_ribo,species_dataset_name,abc_subclass_markercor,Neighborhood,Division,Initial_Class,Initial_Class_markers,Initial_Class_markers_level_2
TAGTATGTAATA_e14-WT8,0.665049,0.042806,e14-WT8-1,ctx,4.120146,3.600646,G2M,Ctx,mouse,14,...,e14-WT8,nan,0.951476,mouse:PRJNA508375,320 Astro-OLF NN,P,NPC,NPC_PEv,NPC_PEv_DLX2,NPC_PEv_DLX2_2
TTCGCATAGGTT_e14-WT8,-0.084758,-0.117966,e14-WT8-1,ctx,3.767379,3.285557,G1,Ctx,mouse,14,...,e14-WT8,nan,0.455775,mouse:PRJNA508375,038 DG-PIR Ex IMN,T,TEd,Ctx_Allo,Ctx_Allo_NR2F2,Ctx_Allo_NR2F2_422
CCTCCAATTGGC_e14-WT8,-0.115723,-0.114059,e14-WT8-1,ctx,4.064121,3.559188,G1,Ctx,mouse,14,...,e14-WT8,nan,0.590896,mouse:PRJNA508375,309 CB PLI Gly-Gaba,T,TEd,Ctx_Allo,Ctx_Allo_NR2F2,Ctx_Allo_NR2F2_161
CACGATATCCAT_e14-WT8,0.273791,-0.049892,e14-WT8-1,ctx,4.045870,3.591176,G2M,Ctx,mouse,14,...,e14-WT8,nan,0.718240,mouse:PRJNA508375,330 VLMC NN,Neighbor,Mesenchyme,VLMC,VLMC,VLMC_211
CAGCCGCGCCCT_e14-WT8,-0.120023,-0.105050,e14-WT8-1,ctx,4.047547,3.553762,G1,Ctx,mouse,14,...,e14-WT8,nan,0.690138,mouse:PRJNA508375,017 CA3 Glut,T,TEd,Ctx_Allo,Ctx_Allo_ZBTB20,Ctx_Allo_ZBTB20_61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_musculus_RNA-Seq_kOut_GTTTCTATCAGTTAGC,-0.147261,-0.100842,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,ctx,2.909556,2.856729,G1,Ctx,mouse,22,...,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,NaN,0.076818,mouse:PRJNA641653,020 L2/3 IT RSP Glut,T,TEd,Ctx_IT,Ctx_IT,Ctx_IT_418
SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_musculus_RNA-Seq_kOut_TGTGTTTGTAGCGTAG,-0.068897,-0.032346,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,ctx,2.974512,2.933487,G1,Ctx,mouse,22,...,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,NaN,0.054289,mouse:PRJNA641653,309 CB PLI Gly-Gaba,T,TEd,Ctx_L5ET,Ctx_L5ET,Ctx_L5ET_576
SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_musculus_RNA-Seq_kOut_TACTCATCACCAGATT,-0.061093,-0.063057,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,ctx,2.945469,2.911690,G1,Ctx,mouse,22,...,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,NaN,0.074301,mouse:PRJNA641653,016 CA1-ProS Glut,T,TEd,Ctx_IT,Ctx_IT,Ctx_IT_89
SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_musculus_RNA-Seq_kOut_GGGATGAGTTCGCGAC,-0.026970,-0.074085,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,ctx,2.903633,2.859138,G1,Ctx,mouse,22,...,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,NaN,0.247218,mouse:PRJNA641653,046 Vip Gaba,T,TEv,CGE_GABA,CGE_GABA,CGE_GABA_357


In [66]:
del adata.obsm['discov_onehot']
del adata.obsm[ 'kmeans_onehot']
del adata.obsm['phase_sex']
del adata.obsm['taxon_probs']

In [79]:
out_reductions='/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/models/DevIntegrations'
obsm_keys=["X_pca", "X_Scanorama", "X_liger", "X_harmony", "X_scVI"]
for x in obsm_keys:
    print(x)
    try:
        adata.obsm[x]=np.loadtxt(os.path.join(out_reductions,x+".csv"), delimiter=",")
        MDE_KEY = x+"_UMAP"
        adata.obsm[MDE_KEY]=np.loadtxt(os.path.join(out_reductions,MDE_KEY+".csv"), delimiter=",")
    except:
        print(x,'FAILED')
obsm_keys = [x for x in obsm_keys if x in adata.obsm.keys()]

X_pca
X_Scanorama
X_Scanorama FAILED
X_liger
X_liger FAILED
X_harmony
X_scVI


In [108]:
adata.obs = adata.obs[keep_columns]

In [109]:
meta_table = adata.obs[meta_columns].drop_duplicates()
meta_table['n_cells'] = meta_table['batch_name'].replace(adata.obs['batch_name'].value_counts().to_dict())
meta_table['mean_log10_n_umis'] = meta_table['batch_name'].replace(adata.obs.groupby('batch_name')['log10_n_counts'].mean().to_dict())
meta_table['mean_log10_n_genes'] = meta_table['batch_name'].replace(adata.obs.groupby('batch_name')['log10_n_genes'].mean().to_dict())
meta_table.to_csv(os.path.join(sc.settings.figdir,'supplement_table_1_batch_meta.tsv'),sep='\t')

/scratch/fast/2574672/ipykernel_2367526/2486640036.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  meta_table['n_cells'] = meta_table['batch_name'].replace(adata.obs['batch_name'].value_counts().to_dict())
/scratch/fast/2574672/ipykernel_2367526/2486640036.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  meta_table['n_cells'] = meta_table['batch_name'].replace(adata.obs['batch_name'].value_counts().to_dict())
/scratch/fast/2574672/ipykernel_2367526/2486640036.py:3: FutureWarning: The default of observed=False is deprecated and will be chang

In [106]:
meta_table

,batch_name,general_region,region,species,timepoint,individual,species_dataset_name,n_cells,mean_log10_n_umis,mean_log10_n_genes
TAGTATGTAATA_e14-WT8,e14-WT8-1,ctx,Ctx,mouse,14,e14-WT8,mouse:PRJNA508375,1092,3.462903,3.186023
GACCCGAGACTA_e14-WT11,e14-WT11,ctx,Ctx,mouse,14,e14-WT11,mouse:PRJNA508375,2480,3.434775,3.172327
ACTACGCTTCGC_p0-WT3,p0-WT3,ctx,Ctx,mouse,21,p0-WT3,mouse:PRJNA508375,2506,3.331296,3.103368
AGGGCATTTGTG_e14-WT10,e14-WT10,ctx,Ctx,mouse,14,e14-WT10,mouse:PRJNA508375,1942,3.437200,3.173372
ACGTTCCCCGTC_p0-WT2,p0-WT2,ctx,Ctx,mouse,21,p0-WT2,mouse:PRJNA508375,2281,3.312711,3.091133
...,...,...,...,...,...,...,...,...,...,...
SRR12082758_GSM4635075_RNA-Seq_E14_5_Mus_musculus_RNA-Seq_kOut_GGAGCAATCTGAGTGT,SRR12082758_GSM4635075_RNA-Seq_E14_5_Mus_muscu...,ctx,Ctx,mouse,14,SRR12082758_GSM4635075_RNA-Seq_E14_5_Mus_muscu...,mouse:PRJNA641653,4194,3.311881,3.265896
SRR12082763_GSM4635080_RNA-Seq_P1_rep1_Mus_musculus_RNA-Seq_kOut_GGCCGATCACAACTGT,SRR12082763_GSM4635080_RNA-Seq_P1_rep1_Mus_mus...,ctx,Ctx,mouse,22,SRR12082763_GSM4635080_RNA-Seq_P1_rep1_Mus_mus...,mouse:PRJNA641653,6432,3.322224,3.275445
SRR12082757_GSM4635074_RNA-Seq_E13_5_Mus_musculus_RNA-Seq_kOut_CAGCTGGTCCACTGGG,SRR12082757_GSM4635074_RNA-Seq_E13_5_Mus_muscu...,ctx,Ctx,mouse,13,SRR12082757_GSM4635074_RNA-Seq_E13_5_Mus_muscu...,mouse:PRJNA641653,7900,3.423231,3.378569
SRR14407607_GSM5277845_P4_Mus_musculus_RNA-Seq_kOut_TCGGGACAGTCCAGGA,SRR14407607_GSM5277845_P4_Mus_musculus_RNA-Seq...,ctx,Ctx,mouse,25,SRR14407607_GSM5277845_P4_Mus_musculus_RNA-Seq...,mouse:PRJNA641653,6297,3.265467,3.220974


In [111]:
adata.write('/home/matthew.schmitz/Matthew/data/taxtest/clean_dev_wb_submit.h5ad')

/home/matthew.schmitz/Matthew/utils/miniforge3/envs/rapids-24.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = c


In [113]:
adata.write('/home/matthew.schmitz/Matthew/data/taxtest/clean_dev_wb_submit_compress.h5ad',compression='gzip')

In [142]:
# get_ipython().system(
#     f'cp /home/matthew.schmitz/Matthew/data/taxtest/20250822_schmitz_hqm_geo_submission/clean_dev_hqm_wb.h5ad '
#     '/allen/programs/celltypes/workgroups/rnaseqanalysis/hct_ux3_cellxgene/anndata_080/V1Evo/'
# )

In [156]:
lvl = "Initial_Class_markers_level_2"
col = "region"

tmp = adata.obs[[lvl, col]].dropna()
cts = (tmp.groupby([lvl, col]).size()
          .rename("n").reset_index())

cts["pct"] = cts["n"] / cts.groupby(lvl)["n"].transform("sum")
cts["rnk"] = cts.groupby(lvl)["n"].rank(method="first", ascending=False)

top3 = (cts.loc[cts["rnk"] <= 3]
          .sort_values([lvl, "n"], ascending=[True, False]))

summary = (top3.groupby(lvl)
               .apply(lambda g: ", ".join(f"{r} {p:.1%}" 
                                          for r, p in zip(g[col], g["pct"])))
               .rename("top3_regions"))

adata.obs["level2_top_regions"] = adata.obs[lvl].map(summary)


/scratch/fast/4048428/ipykernel_2863333/3930376667.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: ", ".join(f"{r} {p:.1%}"


In [159]:
adata.obs[['Initial_Class_markers_level_2','level_2','Neighborhood','Division','Initial_Class','Initial_Class_markers',"level2_top_regions"]].drop_duplicates().sort_values(['Neighborhood','Division','Initial_Class_markers','level_2']).to_csv('/home/matthew.schmitz/WbFigures/DevFinalAnalysis/cluster_meta_table.tsv',sep='\t')

In [161]:
adata.obs[['Neighborhood','Division','Initial_Class','Initial_Class_markers']].drop_duplicates().sort_values(['Neighborhood','Division','Initial_Class_markers']).to_csv('/home/matthew.schmitz/WbFigures/DevFinalAnalysis/initial_class_meta_table.tsv',sep='\t')

In [162]:
adata.obs[['Neighborhood','Division']].drop_duplicates().sort_values(['Neighborhood','Division']).to_csv('/home/matthew.schmitz/WbFigures/DevFinalAnalysis/division_meta_table.tsv',sep='\t')

In [16]:
adata = sc.read_h5ad('/home/matthew.schmitz/Matthew/data/taxtest/geo_submission/clean_dev_hqm_wb_submit_compress.h5ad')

In [ ]:
# log_real_means,real_mean_levels=antipode.model_functions.get_real_leaf_means(adata,discov_key,leaf_key,layer=layer_key)
# real_means=pd.DataFrame(log_real_means.mean(0),columns=adata.var.index,index=real_mean_levels[leaf_key])
# real_means=real_means.loc[real_mean_levels[leaf_key],:]

In [21]:
adata.obs

,G2M_score,S_score,batch_name,general_region,log10_n_counts,log10_n_genes,phase,region,species,timepoint,...,individual,msregion,percent_ribo,species_dataset_name,abc_subclass_markercor,Neighborhood,Division,Initial_Class,Initial_Class_markers,Initial_Class_markers_level_2
TAGTATGTAATA_e14-WT8,0.665049,0.042806,e14-WT8-1,ctx,4.120146,3.600646,G2M,Ctx,mouse,14,...,e14-WT8,nan,0.951476,mouse:PRJNA508375,320 Astro-OLF NN,P,NPC,NPC_PEv,NPC_PEv_DLX2,NPC_PEv_DLX2_2
TTCGCATAGGTT_e14-WT8,-0.084758,-0.117966,e14-WT8-1,ctx,3.767379,3.285557,G1,Ctx,mouse,14,...,e14-WT8,nan,0.455775,mouse:PRJNA508375,038 DG-PIR Ex IMN,T,TEd,Ctx_Allo,Ctx_Allo_NR2F2,Ctx_Allo_NR2F2_422
CCTCCAATTGGC_e14-WT8,-0.115723,-0.114059,e14-WT8-1,ctx,4.064121,3.559188,G1,Ctx,mouse,14,...,e14-WT8,nan,0.590896,mouse:PRJNA508375,309 CB PLI Gly-Gaba,T,TEd,Ctx_Allo,Ctx_Allo_NR2F2,Ctx_Allo_NR2F2_161
CACGATATCCAT_e14-WT8,0.273791,-0.049892,e14-WT8-1,ctx,4.045870,3.591176,G2M,Ctx,mouse,14,...,e14-WT8,nan,0.718240,mouse:PRJNA508375,330 VLMC NN,Neighbor,Mesenchyme,VLMC,VLMC,VLMC_211
CAGCCGCGCCCT_e14-WT8,-0.120023,-0.105050,e14-WT8-1,ctx,4.047547,3.553762,G1,Ctx,mouse,14,...,e14-WT8,nan,0.690138,mouse:PRJNA508375,017 CA3 Glut,T,TEd,Ctx_Allo,Ctx_Allo_ZBTB20,Ctx_Allo_ZBTB20_61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_musculus_RNA-Seq_kOut_GTTTCTATCAGTTAGC,-0.147261,-0.100842,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,ctx,2.909556,2.856729,G1,Ctx,mouse,22,...,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,NaN,0.076818,mouse:PRJNA641653,020 L2/3 IT RSP Glut,T,TEd,Ctx_IT,Ctx_IT,Ctx_IT_418
SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_musculus_RNA-Seq_kOut_TGTGTTTGTAGCGTAG,-0.068897,-0.032346,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,ctx,2.974512,2.933487,G1,Ctx,mouse,22,...,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,NaN,0.054289,mouse:PRJNA641653,309 CB PLI Gly-Gaba,T,TEd,Ctx_L5ET,Ctx_L5ET,Ctx_L5ET_576
SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_musculus_RNA-Seq_kOut_TACTCATCACCAGATT,-0.061093,-0.063057,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,ctx,2.945469,2.911690,G1,Ctx,mouse,22,...,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,NaN,0.074301,mouse:PRJNA641653,016 CA1-ProS Glut,T,TEd,Ctx_IT,Ctx_IT,Ctx_IT_89
SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_musculus_RNA-Seq_kOut_GGGATGAGTTCGCGAC,-0.026970,-0.074085,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,ctx,2.903633,2.859138,G1,Ctx,mouse,22,...,SRR12082764_GSM4635081_RNA-Seq_P1_rep2_Mus_mus...,NaN,0.247218,mouse:PRJNA641653,046 Vip Gaba,T,TEv,CGE_GABA,CGE_GABA,CGE_GABA_357
